In [60]:
import pandas as pd
import numpy as np
import json
import requests
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup as bs
import base64
import datetime
import ticketpy

%matplotlib inline

# Ticketmaster Data

In [88]:
api_keys = pd.read_csv('api_keys.csv')
# git ignore csv for pushing to repo
# .ipynb_checkpoints

In [89]:
# tm_key = api_keys.loc[api_keys['endpoint'] == 'ticketmaster_endpoint', 'key'].values[0]

In [90]:
# tm_endpoint = 'https://app.ticketmaster.com/discovery/v2/events.json?'

In [91]:
# tm_params = {'ver':2.0,'feedtype':'json','api_key':tm_key}

In [92]:
# test = requests.get(tm_endpoint,tm_params)
# test

<Response [401]>

In [93]:
events_request = requests.get('https://app.ticketmaster.com/discovery/v2/events.json?apikey=')
events_request

<Response [200]>

In [94]:
events = events_request.json()
events

{'_embedded': {'events': [{'name': 'Magic Of Lights: Drive-Through Holiday Lights Experience',
    'type': 'event',
    'id': 'vv1AOv_8IGA5_vos',
    'test': False,
    'url': 'https://concerts.livenation.com/magic-of-lights-drivethrough-holiday-lights-holmdel-new-jersey-11-13-2020/event/020059428B641471',
    'locale': 'en-us',
    'images': [{'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/ce5/5a55729f-f567-40cd-92b0-8d64d1209ce5_1181471_RETINA_LANDSCAPE_16_9.jpg',
      'width': 1136,
      'height': 639,
      'fallback': False},
     {'ratio': '16_9',
      'url': 'https://s1.ticketm.net/dam/a/ce5/5a55729f-f567-40cd-92b0-8d64d1209ce5_1181471_TABLET_LANDSCAPE_16_9.jpg',
      'width': 1024,
      'height': 576,
      'fallback': False},
     {'ratio': '3_2',
      'url': 'https://s1.ticketm.net/dam/a/ce5/5a55729f-f567-40cd-92b0-8d64d1209ce5_1181471_RETINA_PORTRAIT_3_2.jpg',
      'width': 640,
      'height': 427,
      'fallback': False},
     {'ratio': '3_2',
      'u

In [95]:
type(events)

dict

In [98]:
event_info = []
for event in events.get('_embedded',{}).get('events',{}):
     event_info.append({
        'TM_id': str(event.get('id',{})),
        'name' : str(event.get('name',{})),
        'artist': list(str(attraction.get('name')) for attraction in event['_embedded'].get('attractions',{})),
        'venue' : list(str(venue.get('name')) for venue in event['_embedded'].get('venues',{}))[0],
        'city' : list(str(venue.get('city',{}).get('name')) for venue in event['_embedded'].get('venues',{}))[0],
        'state' : list(str(venue.get('state',{}).get('stateCode')) for venue in event['_embedded'].get('venues',{}))[0],
        'date' : str(event.get('dates',{}).get('start',{}).get('dateTime',{})),
        'TM_FV_prices': event.get('priceRanges'),
        'genre' : event.get('classifications'),
     })
tmDF = pd.DataFrame(event_info)

In [99]:
tmDF

,TM_id,name,artist,venue,city,state,date,TM_FV_prices,genre
0,vv1AOv_8IGA5_vos,Magic Of Lights: Drive-Through Holiday Lights ...,[Magic Of Lights],PNC Bank Arts Center,Holmdel,NJ,{},"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
1,G5vVZpf7AtYVy,Magic Of Lights: Drive-Through Holiday Lights ...,[Magic Of Lights],Northwell Health at Jones Beach Theater,Wantagh,NY,{},"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
2,vvG1VZp2S5pPb-,Miami Dolphins vs. New England Patriots,"[Miami Dolphins, New England Patriots]",Hard Rock Stadium,Miami,FL,2020-12-20T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
3,vvG1VZpoKLa4fg,Tampa Bay Buccaneers vs. Atlanta Falcons,"[Tampa Bay Buccaneers, Atlanta Falcons]",Raymond James Stadium,Tampa,FL,2021-01-03T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
4,vvG1YZp2XrPfSS,Dallas Cowboys vs. San Francisco 49ers,"[Dallas Cowboys, San Francisco 49ers]",AT&T Stadium,Arlington,TX,2020-12-20T18:00:00Z,None,"[{'primary': True, 'segment': {'id': 'KZFzniwn..."
5,vvG1bZp2BMoOze,Kansas City Chiefs vs. Atlanta Falcons,"[Kansas City Chiefs, Atlanta Falcons]",Arrowhead Stadium,Kansas City,MO,2020-12-27T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
6,vvG1YZ4VLloAHj,Eagles,[Eagles],American Airlines Center,Dallas,TX,2021-09-22T01:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
7,1kk8v94-GA5YE-w,Eagles,[Eagles],Phoenix Suns Arena (formerly Talking Stick Res...,Phoenix,AZ,2021-09-25T03:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
8,G5vzZ4g6yzQHe,Eagles,[Eagles],Ball Arena,Denver,CO,2021-09-19T02:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."
9,vv1AAZA06GkdNls4W,Cleveland Browns vs. Pittsburgh Steelers,"[Cleveland Browns, Pittsburgh Steelers]","FirstEnergy Stadium, Home of the Cleveland Browns",Cleveland,OH,2021-01-03T18:00:00Z,"[{'type': 'standard', 'currency': 'USD', 'min'...","[{'primary': True, 'segment': {'id': 'KZFzniwn..."


# StubHub Data

In [100]:
access_token = 
stubhub_endpoint = 'https://api.stubhub.com/sellers/search/bulk/events/v3'
headers = {'Authorization': 'Bearer' + access_token, 'Accept': 'application/json', 'Accept-Encoding':'gzip'}

In [101]:
stubhub_request = requests.get(stubhub_endpoint,headers)
stubhub_request

<Response [401]>